# **Compliance GPT with LangChain**

## **Setup**

In [1]:
# import nest_asyncio
# nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

## **Config**

In [2]:
from utils.config import get_config

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'conv_1'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [5]:
from database.vector_store.vector_store import RedisIndexManager, PineconeIndexManager, PostgresIndexManager, ElasticIndexManager

es_bi = ElasticIndexManager(index_name='bi', embed_model=embed_model, config=config)
vector_store_bi = es_bi.load_vector_index()

es_ojk = ElasticIndexManager(index_name='ojk', embed_model=embed_model, config=config)
vector_store_ojk = es_ojk.load_vector_index()

es_ket = ElasticIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)
vector_store_ket = es_ket.load_vector_index()

es_rek = ElasticIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
vector_store_rek = es_rek.load_vector_index()

# postgres_bi = PostgresIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = postgres_bi.load_vector_index()

# postgres_ojk = PostgresIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = postgres_ojk.load_vector_index()

# postgres_ket = PostgresIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = postgres_ket.load_vector_index()

# postgres_rek = PostgresIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = postgres_rek.load_vector_index()

# redis_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
# vector_store_bi = redis_bi.load_vector_index()

# redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
# vector_store_ojk = redis_ojk.load_vector_index()

# redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
# vector_store_ket = redis_sikepo_ket.load_vector_index()

# redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
# vector_store_rek = redis_sikepo_rek.load_vector_index()

# pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = pinecone_ojk.load_vector_index()

# pinecone_bi = PineconeIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = pinecone_bi.load_vector_index()

# pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = pinecone_ket.load_vector_index()

# pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = pinecone_rek.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc-new\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use Use class in langchain-elasticsearch package instead.
  warn_deprecated(


Loaded index 'bi'.
Loaded index 'ojk'.
Loaded index 'sikepo-rekam-jejak'.
Loaded index 'sikepo-ketentuan-terkait'.


In [6]:
from database.vector_store.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [7]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

top_n = 5

retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=top_n,
                                  llm_model=llm_model, embed_model=embed_model, config=config)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_n=top_n,
                                llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, top_n=top_n,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, top_n=top_n,
                                   llm_model=llm_model, embed_model=embed_model, config=config)

In [8]:
# retriever_bi.invoke("What is the maximum transaction of QRIS?")

## **Create Chain**

In [9]:
from database.chat_store import ElasticChatStore

chat_store = ElasticChatStore(config=config, k=3)
# chat_store = PostgresChatStore(config=config, k=3)
# chat_store = RedisChatStore(k=3, config=config, db_id=1)
# chat_store = MongoDBChatStore(config=config, k=4)

In [10]:
from chain.rag_chain import create_chain_with_chat_history, create_sequential_chain
from chain.chain_sikepo.graph_cypher_sikepo_chain import graph_rag_chain

graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_sequential_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_sikepo_ket,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [11]:
# to print chat history
# print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

In [12]:
# chat_store.get_conversation_ids_by_user_id(user_id=USER_ID)

In [13]:
# chat_store.get_conversation(user_id=USER_ID, conversation_id=CONVERSATION_ID)

In [18]:
chat_store.get_conversation_ids_by_user_id(user_id='xmriz')

[{'conversation_id': 'conv_1', 'title': 'Apa itu dana hibah?...'},
 {'conversation_id': 'conv_2', 'title': 'Apa itu dana tabarru?...'}]

In [ ]:
# chat_store.clear_session_history_by_userid(user_id='budi')

In [ ]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [17]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Siapa lionel messi?",
    user_id=USER_ID,
    conversation_id="conv_1"
)

response

{'rewrited question': 'Apa itu dana tabarru?',
 'answer': 'Dana tabarru adalah dana yang dikumpulkan oleh perusahaan asuransi syariah dari peserta atau pemegang polis untuk digunakan dalam membayar santunan atau klaim yang timbul akibat musibah sesuai dengan prinsip-prinsip syariah.\n\nSource: [10/SEOJK.05/2015](https://www.ojk.go.id/id/regulasi/Documents/Pages/SEOJK-tentang-Pedoman-Pembentukan-Penyisihan-Kontribusi-dan-Metode-Perhitungan-Penyisihan-Klaim-pada-Usaha-asuransi-Syariah/SEOJK%2010.%20%20Metode%20Perhitungan%20Penyisihan%20Klaim%20%20Asuransi%20Syariah.pdf#page=3)',
 'context': [Document(metadata={'doc_id': 1365, 'title': 'SEOJK tentang Pedoman Pembentukan Penyisihan Kontribusi dan Metode Perhitungan Penyisihan Klaim pada Usaha asuransi Syariah atau Reasuransi Syariah', 'sector': 'IKNB', 'subsector': 'IKNB Syariah,  Asuransi', 'regulation_type': 'Surat Edaran OJK', 'regulation_number': '10/SEOJK.05/2015', 'effective_date': '2015/04/17', 'file_url': 'https://www.ojk.go.id/id

In [ ]:
# print(response['answer'])


In [ ]:
# print(response['context'])

#### **Streaming**

In [ ]:
# from chain.rag_chain import print_answer_stream

# question = "Jika saya ingin membuat peraturan terkait bank digital peraturan apa yang perlu saya perhatikan?"

# print_answer_stream(chain=chain_history, question=question, user_id=USER_ID, conversation_id=CONVERSATION_ID)

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('76b9a325.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Jika Anda ingin membuat peraturan terkait bank digital, Anda perlu memperhatikan beberapa peraturan yang relevan, antara lain:

1. **Peraturan Otoritas Jasa Keuangan (POJK) Nomor 21 Tahun 2023** tentang Layanan Digital oleh Bank Umum. Peraturan ini mengatur tentang pengembangan dan inovasi dalam memberikan layanan digital oleh bank, serta manajemen risiko, keamanan data nasabah, dan perlindungan konsumen. 

   Source: [21 Tahun 2023](https://www.ojk.go.id/id/regulasi/Documents/Pages/Layanan-Digital-oleh-Bank-Umum/POJK%2021%20Tahun%202023.%20Layanan%20Digital%20oleh%20Bank%20Umum.pdf) (halaman 1)

2. **Peraturan Otoritas Jasa Keuangan (POJK) Nomor 12/POJK.03/2018** tentang Penyelenggaraan Layanan Perbankan Digital oleh Bank Umum. Peraturan ini perlu diperhatikan karena akan digantikan oleh POJK 21 Tahun 2023, namun masih relevan untuk memahami ketentuan yang ada sebelumnya.

   Source: [12/POJK.03/2018](https://www.ojk.go.id/id/regulasi/Documents/Pages/Penyelenggaraan-Layanan-Perbankan-